<a href="https://colab.research.google.com/github/takashisuo/signate/blob/master/bento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor as RFR

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train_df = pd.read_csv("./drive/My Drive/Colab Notebooks/signate/train_bento.csv", index_col="datetime")
test_df =  pd.read_csv("./drive/My Drive/Colab Notebooks/signate/test_bento.csv", index_col="datetime")

In [0]:
train_df.shape, test_df.shape
trainY = train_df.loc[:, "y"]
train_df = train_df.drop("y", axis=1)

In [6]:
train_df.shape, test_df.shape

((207, 10), (40, 10))

In [7]:
# merge train_df and test_df because expand column
merge_df = pd.concat([train_df, test_df], axis=0)
merge_df.shape

(247, 10)

In [8]:
merge_df = pd.get_dummies(merge_df)
merge_df.shape

(247, 213)

In [9]:
merge_df.isna().sum()

soldout                               0
kcal                                 45
payday                              235
temperature                           0
week_月                                0
week_木                                0
week_水                                0
week_火                                0
week_金                                0
name_いか天ぷら                            0
name_かじきの甘辛煮                          0
name_きのこソースハンバーグ                      0
name_さっくりメンチカツ                        0
name_さわら焼味噌掛け                         0
name_さんま辛味焼                           0
name_たっぷりベーコンフライ                      0
name_なすと挽肉のはさみ揚げ                      0
name_ひやしたぬきうどん・炊き込みご飯                 0
name_ぶりレモンペッパー焼き                      0
name_ますのマスタードソース                      0
name_やわらかロースのサムジョン                    0
name_アジ唐揚げ南蛮ソース                       0
name_エビフライ                            0
name_エビフライ・エビカツ                       0
name_カキフライタルタル                        0


In [0]:
def drop_attribute(data):
    data = data.drop('kcal', axis=1)
    data = data.drop('payday', axis=1)
    return data

In [0]:
merge_df = drop_attribute(merge_df)

In [12]:
merge_df.head()

,soldout,temperature,week_月,week_木,week_水,week_火,week_金,name_いか天ぷら,name_かじきの甘辛煮,name_きのこソースハンバーグ,name_さっくりメンチカツ,name_さわら焼味噌掛け,name_さんま辛味焼,name_たっぷりベーコンフライ,name_なすと挽肉のはさみ揚げ,name_ひやしたぬきうどん・炊き込みご飯,name_ぶりレモンペッパー焼き,name_ますのマスタードソース,name_やわらかロースのサムジョン,name_アジ唐揚げ南蛮ソース,name_エビフライ,name_エビフライ・エビカツ,name_カキフライタルタル,name_カレイの唐揚げ,name_カレイ唐揚げ 甘酢あん,name_カレイ唐揚げ夏野菜あん,name_カレイ唐揚げ野菜あんかけ,name_カレイ唐揚げ野菜餡かけ,name_カレイ野菜あんかけ,name_カレー入りソーセージカツ,name_キスと野菜の天ぷら,name_キーマカレー,name_クリームチーズ入りメンチ,name_ゴーヤチャンプルー,name_サバ焼味噌掛け,name_サーモンのムニエル2色ソース,name_サーモンフライ・タルタル,name_ジャンボチキンカツ,name_ジューシーメンチカツ,name_スタミナ炒め,...,name_鶏の唐揚げ甘酢あん,name_鶏の塩から揚げ,name_鶏の天ぷら,name_鶏の照り焼きマスタード,name_鶏の照り焼きマヨ,name_鶏の親子煮,name_鶏チリソース,name_鶏肉とカシューナッツ炒め,name_鶏肉と野菜の黒胡椒炒め,name_鶏肉のカレー唐揚,name_鶏肉のスイートチリソース,name_鶏肉の唐揚げ,name_鶏肉の山賊焼き,name_鶏肉の治部煮風,name_麻婆春雨,name_麻婆豆腐,remarks_お楽しみメニュー,remarks_スペシャルメニュー（800円）,remarks_手作りの味,remarks_料理長のこだわりメニュー,remarks_近隣に飲食店複合ビルオープン,remarks_酢豚（28食）、カレー（85食）,remarks_鶏のレモンペッパー焼（50食）、カレー（42食）,event_キャリアアップ支援セミナー,event_ママの会,weather_快晴,weather_晴れ,weather_曇,weather_薄曇,weather_雨,weather_雪,weather_雷電,precipitation_--,precipitation_0,precipitation_0.5,precipitation_1,precipitation_1.5,precipitation_2.5,precipitation_6,precipitation_6.5
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-11-18,0,19.8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2013-11-19,1,17.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2013-11-20,0,15.5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2013-11-21,1,15.2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2013-11-22,1,16.1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [13]:
from sklearn.preprocessing import StandardScaler

ssmodel = StandardScaler()
ssmodel.fit(merge_df)
merge_df_scaled = ssmodel.transform(merge_df)
merge_df_scaled.shape

(247, 211)

In [14]:
# divide to run model
trainX = merge_df_scaled[:207, :]
testX = merge_df_scaled[207:, :]
trainX.shape, testX.shape


((207, 211), (40, 211))

In [20]:
from sklearn.model_selection import GridSearchCV
search_params = {
    'n_estimators'      : [5, 10, 20, 30, 50, 100],
    'max_features'      : [10, 50, 100, 150, trainX.shape[1]],
    'random_state'      : [2525],
    'n_jobs'            : [5],
    'min_samples_split' : [3, 5, 10, 15, 20, 25, 30],
    'max_depth'         : [3, 5, 10, 15, 20, 25, 30]
}

gsr = GridSearchCV(
    RFR(),
    search_params,
    cv = 3,
    n_jobs = -1,
    verbose=True
)
 
gsr.fit(trainX, trainY)

Fitting 3 folds for each of 1470 candidates, totalling 4410 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   21.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   49.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 4410 out of 4410 | elapsed:  8.2min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': [3, 5, 10, 15, 20, 2

In [21]:
gsr.best_params_

{'max_depth': 15,
 'max_features': 100,
 'min_samples_split': 5,
 'n_estimators': 5,
 'n_jobs': 5,
 'random_state': 2525}

In [0]:
pred = gsr.predict(testX)

In [0]:
# register
submit = pd.DataFrame()
submit[0] = test_df.index # insert id to variable
submit[1] = pred # insert predict to variable

In [25]:
submit

,0,1
0,2014-10-1,59.261905
1,2014-10-2,57.324887
2,2014-10-3,48.653459
3,2014-10-6,74.431429
4,2014-10-7,59.226316
5,2014-10-8,57.934286
6,2014-10-9,57.379649
7,2014-10-10,106.133333
8,2014-10-14,56.600000
9,2014-10-15,120.910000


In [0]:
submit.to_csv("./drive/My Drive/Colab Notebooks/signate/submit.csv", header=None, index=None)